In [1]:
# Cell 1: Setup & Install
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [2]:
# Install required libraries
!pip install ultralytics python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 25.9 MB/s eta 0:00:00


In [4]:
# Import necessary modules
from ultralytics import YOLO
from docx import Document
import os
import zipfile

# =====================
# Unzip datasets
# =====================
dataset_paths = {
    'day': '/content/drive/MyDrive/dataset/data/carla_day.v1i.yolov8.zip',
    'night': '/content/drive/MyDrive/dataset/data/carla_night.v1i.yolov8.zip',
    'day_rain': '/content/drive/MyDrive/dataset/data/carla_day_rain.v1i.yolov8.zip',
    'night_rain': '/content/drive/MyDrive/dataset/data/carla_night_rain.v1i.yolov8.zip'
}

for dataset_name, zip_path in dataset_paths.items():
    os.makedirs(f'/content/datasets/{dataset_name}', exist_ok=True)
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(f'/content/datasets/{dataset_name}')

# =====================
# Models to compare
# =====================
models = {
    # 'YOLOv8': 'yolov8n.yaml',   # Train from scratch
    # 'YOLOv9': 'yolov9c.yaml',   # Adjust if available
    'YOLOv10': 'yolov10n.yaml'
    # 'YOLOv11': 'yolov11n.yaml'
}

results_dict = {}
output_base = '/content/outputs'
os.makedirs(output_base, exist_ok=True)

# =====================
# Train + Evaluate Loop
# =====================
for dataset_name in dataset_paths:
    data_yaml = f'/content/datasets/{dataset_name}/data.yaml'
    results_dict[dataset_name] = {}

    for model_name, model_cfg in models.items():
        print(f"\nTraining and evaluating {model_name} on {dataset_name}...\n")

        combo_folder = f'{output_base}/{dataset_name}_{model_name}'
        os.makedirs(combo_folder, exist_ok=True)

        # Initialize new model (no pretrained weights)
        model = YOLO(model_cfg)

        # Train
        model.train(
            data=data_yaml,
            epochs=30,
            imgsz=640,
            batch=16,
            device=0,
            project=output_base,
            name=f'{dataset_name}_{model_name}_train'
        )

        # Validate
        results = model.val(
            project=output_base,
            name=f'{dataset_name}_{model_name}_val',
            save=True,
            save_txt=True,
            save_conf=True,
            save_json=True
        )

        # Metrics
        map50 = results.box.map50
        map = results.box.map
        precision = results.box.mp
        recall = results.box.mr
        inference_time_ms = results.speed['inference']
        fps = 1000 / inference_time_ms if inference_time_ms > 0 else 0

        results_dict[dataset_name][model_name] = {
            'mAP50': map50,
            'mAP': map,
            'Precision': precision,
            'Recall': recall,
            'FPS': fps
        }

        # Save metrics
        metrics_txt = f'{combo_folder}/metrics.txt'
        with open(metrics_txt, 'w') as f:
            f.write(f"mAP50: {map50}\n")
            f.write(f"mAP: {map}\n")
            f.write(f"Precision: {precision}\n")
            f.write(f"Recall: {recall}\n")
            f.write(f"FPS: {fps}\n")

        print(f"{model_name} on {dataset_name} Results:")
        print(f"mAP50: {map50}")
        print(f"mAP: {map}")
        print(f"Precision: {precision}")
        print(f"Recall: {recall}")
        print(f"FPS: {fps}")
        print(f"Outputs saved in: {combo_folder}")

# =====================
# Save Results to Word
# =====================
doc = Document()
doc.add_heading('YOLO Models Comparison Across Datasets (Trained from Scratch)', 0)

for dataset_name, model_results in results_dict.items():
    doc.add_heading(f'Results for {dataset_name} Dataset', level=1)

    table = doc.add_table(rows=1, cols=6)
    table.style = 'Table Grid'

    hdr_cells = table.rows[0].cells
    hdr_cells[0].text = 'Model'
    hdr_cells[1].text = 'mAP50'
    hdr_cells[2].text = 'mAP (0.5:0.95)'
    hdr_cells[3].text = 'Precision'
    hdr_cells[4].text = 'Recall'
    hdr_cells[5].text = 'FPS (approx)'

    for model_name, metrics in model_results.items():
        row_cells = table.add_row().cells
        row_cells[0].text = model_name
        row_cells[1].text = f"{metrics['mAP50']:.4f}"
        row_cells[2].text = f"{metrics['mAP']:.4f}"
        row_cells[3].text = f"{metrics['Precision']:.4f}"
        row_cells[4].text = f"{metrics['Recall']:.4f}"
        row_cells[5].text = f"{metrics['FPS']:.2f}"

doc.add_paragraph('\nNotes:')
doc.add_paragraph('- mAP50: mean Average Precision at IoU=0.5')
doc.add_paragraph('- mAP: mean Average Precision at IoU=0.5 to 0.95')
doc.add_paragraph('- Precision and Recall are averaged over classes')
doc.add_paragraph('- FPS is approximate, based on validation inference time')
doc.add_paragraph('- Models trained from scratch; performance may improve with more epochs or a larger dataset')
doc.add_paragraph('- Outputs (predicted images, labels, metrics.txt) are in /content/outputs/{dataset_model} folders')

doc_path = '/content/yolo_comparison_all_datasets.docx'
doc.save(doc_path)

print(f"\nResults saved to {doc_path}. Download it from Colab files.")
print(f"All outputs organized in /content/outputs/ - You can browse/download from Colab's file explorer.")

# =====================
# Zip + Download
# =====================
!zip -r /content/outputs.zip /content/outputs
from google.colab import files
files.download('/content/outputs.zip')
files.download(doc_path)



Training and evaluating YOLOv10 on day...

Ultralytics 8.3.182 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/datasets/day/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov10n.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=day_YOLOv10_train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patie

Overriding model.yaml nc=80 with nc=4

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1      9856  ultralytics.nn.modules.block.SCDown          [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1     36096  ultralytics

YOLOv10n summary: 223 layers, 2,708,600 parameters, 2,708,584 gradients, 8.4 GFLOPs

Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2157.9±775.1 MB/s, size: 85.1 KB)


train: Scanning /content/datasets/day/train/labels... 2100 images, 42 backgrounds, 0 corrupt: 100%|██████████| 2100/2100 [00:00<00:00, 2484.54it/s]

...

                   all        190        340      0.861        0.7      0.797      0.689
                   car        143        158      0.892      0.911      0.967      0.902
            pedestrian         61         67      0.786      0.731      0.818      0.596
        traffic ligths          7          7          1          0      0.328      0.227
    traffic sign board         25         25      0.958      0.907      0.917      0.771
                 truck         81         83      0.667      0.952      0.957      0.951
Speed: 0.2ms preprocess, 6.1ms inference, 0.0ms loss, 0.2ms postprocess per image
Saving /content/outputs/night_rain_YOLOv10_val/predictions.json...
Results saved to /content/outputs/night_rain_YOLOv10_val
YOLOv10 on night_rain Results:
mAP50: 0.7971732025704619
mAP: 0.6893313011816463
Precision: 0.860518990085609
Recall: 0.7003986288994618
FPS: 164.96178345098946
Outputs saved in: /content/outputs/night_rain_YOLOv10

Results saved to /content/yolo_comparison_al

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>